In [1]:
import scrapy
from scrapy import Selector
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
import requests

In [2]:
class LatLibSpider(scrapy.Spider):
    name = 'latlib_spider'
    def start_requests(self):
        urls = ['https://www.thelatinlibrary.com/']
        for url in urls:
            yield scrapy.Request( url = url, callback = self.parse_front)
    def parse_front(self, response):
        #opening latinlibrary page
        author_links = response.xpath('.//table[1]/tr/td/a/@href')
        links_to_follow = author_links.extract()
        for url in links_to_follow:
            yield response.follow(url = url, callback = self.parse_authors)
            
    def parse_authors(self, response):
        #navigate to each author page
        author_page = response.xpath('//div[@class="work"]/tbody/tr/td/a/@href')
        work_links = author_page.extract()
        for url in work_links:
            yield response.follow(url = url, callback = self.parse_works)

    def parse_works(self, response):
        #get the content of each work under the authors
        #save to txt files
        #title
        work_title = response.xpath('//title/text()')
        work_title_ext = work_title.extract()
        #body--now I realize I will just scrape everything and not focus on the specific types of body text (disregard 2+3)
        body_text = response.xpath('//body/text()').extract() 
        #body_text2 = response.xpath('//body/*/text()').extract() #like Cato
        #body_text3 = response.xpath('//p/a[@name="#(need to iterate through numbers here)"]').extract() #like Suetonius
        
        html_file = 'lat_texts.html'
        #with open(html_file, 'w') as f:
           # f.write( texts for texts in body_text1 )
            
        with open(html_file, 'wb') as fout:
            fout.write(response.body)
        
        


In [3]:
process = CrawlerProcess()
process.crawl(LatLibSpider)
process.start()

2021-03-04 14:42:28 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-04 14:42:28 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 (default, Sep  4 2020, 02:22:02) - [Clang 10.0.0 ], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform macOS-10.15.5-x86_64-i386-64bit
2021-03-04 14:42:28 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-04 14:42:28 [scrapy.crawler] INFO: Overridden settings:
{}
2021-03-04 14:42:28 [scrapy.extensions.telnet] INFO: Telnet Password: 6002f3c16fadce4e
2021-03-04 14:42:28 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-03-04 14:42:28 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddle